In [83]:
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
from keras.models import Sequential, Model
from keras.layers import Input, Activation, GRU, Dense
from sklearn.metrics import mean_squared_error as mse
from sklearn.preprocessing import scale, StandardScaler, RobustScaler
from collections import OrderedDict, defaultdict, Counter
plt.rcParams['figure.figsize'] = [10, 8]

### Traditional Caching Algorithms

### LRU

In [84]:
class LruContentStore():
    def __init__(self, size):
        self.size = size
        self.store = OrderedDict()
        self.hits = 0
        self.misses = 0

    def add(self, item):
        if self.size:
            if(len(self.store) == self.size):
                self.store.popitem(last=False)
            self.store[item] = item

    def get(self, item):
        try:
            cached_item = self.store.pop(item)
            self.store[item] = cached_item
            self.hits += 1
            return cached_item
        except:
            self.misses += 1
            return None
    
    def refresh(self):
        pass

### LFU

In [85]:
class LfuContentStore():
    def __init__(self, size):
        self.size = size
        self.store = {} # {'name', [item, freq]}
        self.hits = 0
        self.misses = 0
    
    def add(self, item):
        if self.size:
            if len(self.store) == self.size:
                min_key = None
                min_freq = None
                for key in self.store.keys():
                    if min_freq == None or self.store[key][1] < min_freq:
                        min_freq = self.store[key][1]
                        min_key = key
                self.store.pop(min_key)
            self.store[item] = [item, 1]

    def get(self, item):
        try:
            cached_item = self.store[item][0]
            self.store[item][1] += 1
            self.hits += 1
            return cached_item
        except:
            self.misses += 1
            return None
    
    def refresh(self):
        pass

### Random

In [86]:
class RandomContentStore():
    def __init__(self, size):
        self.rng = np.random.RandomState(123)
        self.size = size
        self.store = {}
        self.hits = 0
        self.misses = 0

    def add(self, item):
        if self.size:
            if len(self.store) == self.size:
                self.store.pop(self.rng.choice(list(self.store.keys())))
            self.store[item] = item
    
    def get(self, item):
        try:
            cached_item = self.store[item]
            self.hits += 1
            return cached_item
        except:
            self.misses += 1
            return None
    
    def refresh(self):
        pass

### Rolling Moving Average

In [87]:
class RMAContentStore():
    def __init__(self, size):
        self.size = size
        self.store = {}
        self.hits = 0
        self.misses = 0
        self.history = {}
        self.ranking = defaultdict(int)
        self.alpha = 0.1
        self.interval_count = 0
        self.window = 7
        
    def add(self, item):
        if item not in self.history:
            self.history[item] = [0 for _ in range(self.window)]
        self.history[item][self.interval_count % 7] += 1
        if self.size:
            if len(self.store) == self.size:
                min_key, min_rank = self.get_min()
                if min_rank != None and min_rank < self.ranking[item]:
                    self.store.pop(min_key)
                    self.store[item] = item
            else:
                self.store[item] = item
    
    def get_min(self):
        min_key = None
        min_rank = None
        for key in self.ranking.keys():
            if min_key == None or self.ranking[key] < min_rank:
                min_rank = self.ranking[key]
                min_key = key
        return min_key, min_rank
    
    def refresh(self):
        self.interval_count += 1
        for key in self.ranking.keys():
            self.ranking[key] = sum(self.history[key])/min(self.interval_count, self.window)
        
    def get(self, item):
        try:
            cached_item = self.store[item]
            self.hits += 1
            return cached_item
        except:
            self.misses += 1
            return None

### Exponential Moving Average

In [88]:
class EMAContentStore():
    def __init__(self, size):
        self.size = size
        self.store = {}
        self.hits = 0
        self.misses = 0
        self.history = defaultdict(int)
        self.ranking = defaultdict(int)
        self.alpha = 0.1
        
    def add(self, item):
        self.history[item] += 1
        if self.size:
            if len(self.store) == self.size:
                min_key, min_rank = self.get_min()
                if min_rank != None and min_rank < self.ranking[item]:
                    self.store.pop(min_key)
                    self.store[item] = item
            else:
                self.store[item] = item
    
    def get_min(self):
        min_key = None
        min_rank = None
        for key in self.ranking.keys():
            if min_key == None or self.ranking[key] < min_rank:
                min_rank = self.ranking[key]
                min_key = key
        return min_key, min_rank
    
    def refresh(self):
        for key in self.ranking.keys():
            self.ranking[key] = self.ranking[key] + self.alpha*(self.history[key]-self.ranking[key])
        self.history = defaultdict(int)
        
    def get(self, item):
        try:
            cached_item = self.store[item]
            self.hits += 1
            return cached_item
        except:
            self.misses += 1
            return None

### One-day Lookback

In [90]:
class ODContentStore():
    def __init__(self, size):
        self.size = size
        self.store = {}
        self.hits = 0
        self.misses = 0
        self.history = defaultdict(int)
        self.ranking = {}
        
    def add(self, item):
        self.history[item] += 1
        if self.size:
            if len(self.store) == self.size:
                min_key, min_rank = self.get_min()
                if min_rank != None and min_rank < self.ranking[item]:
                    self.store.pop(min_key)
                    self.store[item] = item
            else:
                 self.store[item] = item

    def get_min(self):
        min_key = None
        min_rank = None
        for key in self.ranking.keys():
            if min_key == None or self.ranking[key] < min_rank:
                min_rank = self.ranking[key]
                min_key = key
        return min_key, min_rank
                    
    def refresh(self):
        self.ranking = self.history.copy()
        self.history = defaultdict(int)
        
    def get(self, item):
        try:
            cached_item = self.store[item]
            self.hits += 1
            return cached_item
        except:
            self.misses += 1
            return None

### Init Content Stores

In [100]:
cache_size = int(0.01 * test_set.shape[0])
lru = LruContentStore(cache_size)
lfu  = LfuContentStore(cache_size)
rand = RandomContentStore(cache_size)
rma = RMAContentStore(cache_size)
ema = EMAContentStore(cache_size)
od = ODContentStore(cache_size)
cses = [lru, lfu, rand, rma, ema, od]

### Initialize Statistical Baselines with Training Set

In [101]:
train_set = np.load('train_set.npy')

In [102]:
# initialize OD with training data
yesterday = train_set[:, -1, 0]
for i, val in enumerate(yesterday):
    od.ranking[i] = val

In [103]:
# initialize SMA with training data
last_week = train_set[:, -7:, 0]
last_week_avg = np.mean(last_week, axis=1)
for i, val in enumerate(last_week_avg):
    rma.ranking[i] = val

In [104]:
# initialize EMA with training data
history = train_set[:, :, 0]
for i in range(history.shape[0]):
    ema.ranking[i] = history[i, 0]
for i in range(1, history.shape[1]):
    recent_history = history[:, i]
    for j in range(history.shape[0]): 
        ema.ranking[j] = ema.ranking[j] + ema.alpha*(ema.ranking[j]-recent_history[j])

### Run Tests with Test Set

In [105]:
test_set = np.load('test_set.npy')

In [106]:
seed = 123
for cs in cses:
    np.random.seed(123)
    for i in range(test_set.shape[1]):
        daily_reqs = test_set[:, i, 0]
        flat_daily_reqs = np.repeat(np.arange(test_set.shape[0], dtype='float'), 
                                    daily_reqs.astype('int'), 
                                    axis=0)
        np.random.shuffle(flat_daily_reqs)
        for req in flat_daily_reqs:
            if cs.get(req) == None:
                cs.add(req)
    print(cs.hits/(cs.hits+cs.misses))

0.3692767690443395


KeyboardInterrupt: 